<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/mistral_agent_predictive_Maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
!pip install mistralai -q

In [1]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Failure Prediction Agent's response format
class MaintenancePredictionResult(BaseModel):
    component_id: str
    predicted_failure_date: str
    confidence_score: float
    recommendations: str
    flagged_metrics: dict

print("Creating AI agents for Predictive Maintenance Domain...")

# --- Agent Definitions (Predictive Maintenance Domain) ---

# 1. Equipment Monitoring Agent (Refactored from Customer Onboarding Agent)
equipment_monitoring_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for collecting and processing real-time sensor data and status from industrial equipment.",
    name="equipment-monitoring-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_sensor_data",
                "description": "Retrieve current sensor readings (e.g., temperature, vibration, pressure) for a specific equipment component.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Unique identifier for the equipment (e.g., 'Pump-001')."},
                        "sensor_type": {"type": "string", "description": "Type of sensor data to retrieve (e.g., 'vibration', 'temperature')."}
                    },
                    "required": ["equipment_id", "sensor_type"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_equipment_status",
                "description": "Log the current operational status of a piece of equipment (e.g., 'running', 'idle', 'error').",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Unique identifier for the equipment."},
                        "status": {"type": "string", "description": "Current operational status of the equipment."}
                    },
                    "required": ["equipment_id", "status"]
                }
            }
        }
    ]
)
print(f"Equipment Monitoring Agent '{equipment_monitoring_agent.name}' created with ID: {equipment_monitoring_agent.id}")

# 2. Anomaly Detection Agent (Refactored from Financial Research Agent)
anomaly_detection_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for identifying unusual patterns or anomalies in equipment performance data that may indicate impending issues.",
    name="anomaly-detection-agent",
    tools=[
        {"type": "web_search"}, # For searching external anomaly detection best practices.
        {
            "type": "function",
            "function": {
                "name": "detect_anomalies",
                "description": "Analyze recent equipment performance data for unusual spikes, drops, or patterns.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Unique identifier for the equipment."},
                        "data_series_json": {"type": "string", "description": "JSON string of performance data over time, e.g., '[{\"timestamp\": \"...\", \"value\": 10.5}]'."}
                    },
                    "required": ["equipment_id", "data_series_json"]
                }
            }
        }
    ]
)
print(f"Anomaly Detection Agent '{anomaly_detection_agent.name}' created with ID: {anomaly_detection_agent.id}")

# 3. Failure Prediction Agent (Refactored from Transaction Analyzer Agent)
failure_prediction_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="failure-prediction-agent",
    description="Agent used to predict the likelihood and timing of component failures based on historical data and real-time metrics.",
    instructions="Predict potential component failures, provide a confidence score, and suggest preventative actions.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "maintenance_prediction_result",
                "schema": MaintenancePredictionResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_component_failure",
                "description": "Predict the failure date for a given equipment component based on its current health metrics.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "component_id": {"type": "string", "description": "Identifier of the component (e.g., 'Motor-A')."},
                        "current_health_metrics_json": {"type": "string", "description": "JSON string of current health metrics, e.g., '{\"vibration\": 0.8, \"temp\": 75}'."}
                    },
                    "required": ["component_id", "current_health_metrics_json"]
                }
            }
        }
    ]
)
print(f"Failure Prediction Agent '{failure_prediction_agent.name}' created with ID: {failure_prediction_agent.id}")

# 4. Maintenance Scheduling Agent (Refactored from Loan Management Agent)
maintenance_scheduling_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for optimizing and scheduling maintenance tasks based on failure predictions, resource availability, and operational impact.",
    name="maintenance-scheduling-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "schedule_preventative_maintenance",
                "description": "Schedule preventative maintenance for a component on a specific date, considering predicted failure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "component_id": {"type": "string", "description": "Identifier of the component."},
                        "scheduled_date": {"type": "string", "description": "Proposed date for maintenance in ISO-MM-DD format."},
                        "priority": {"type": "string", "description": "Priority of the maintenance (e.g., 'high', 'medium', 'low')."}
                    },
                    "required": ["component_id", "scheduled_date", "priority"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_maintenance_team_availability",
                "description": "Check the availability of maintenance teams or technicians for a given date.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "team_type": {"type": "string", "description": "Type of maintenance team (e.g., 'electrical', 'mechanical')."},
                        "date": {"type": "string", "description": "Date to check availability in ISO-MM-DD format."}
                    },
                    "required": ["team_type", "date"]
                }
            }
        }
    ]
)
print(f"Maintenance Scheduling Agent '{maintenance_scheduling_agent.name}' created with ID: {maintenance_scheduling_agent.id}")


# 5. Parts Inventory Agent (Refactored from Fraud Detection Agent)
parts_inventory_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing spare parts inventory, checking stock levels, and initiating reorders based on predicted needs.",
    name="parts-inventory-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_part_stock",
                "description": "Check the current stock level for a specific spare part.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "part_number": {"type": "string", "description": "Unique part number (e.g., 'XYZ-456')."}
                    },
                    "required": ["part_number"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "order_spare_part",
                "description": "Place an order for a specific spare part from a supplier.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "part_number": {"type": "string", "description": "Unique part number."},
                        "quantity": {"type": "number", "description": "Quantity to order."},
                        "supplier_id": {"type": "string", "description": "ID of the supplier."}
                    },
                    "required": ["part_number", "quantity", "supplier_id"]
                }
            }
        }
    ]
)
print(f"Parts Inventory Agent '{parts_inventory_agent.name}' created with ID: {parts_inventory_agent.id}")


# 6. Maintenance History Agent (Refactored from Branch Operations Agent)
maintenance_history_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for accessing and analyzing historical maintenance records and repair logs for equipment.",
    name="maintenance-history-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_equipment_maintenance_history",
                "description": "Retrieve all past maintenance records for a specific piece of equipment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Unique identifier for the equipment."}
                    },
                    "required": ["equipment_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_repair_logs",
                "description": "Search repair logs for specific issues or components across multiple equipment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {"type": "string", "description": "Keywords to search in repair logs (e.g., 'bearing noise', 'motor overheating')."}
                    },
                    "required": ["search_query"]
                }
            }
        }
    ]
)
print(f"Maintenance History Agent '{maintenance_history_agent.name}' created with ID: {maintenance_history_agent.id}")


# 7. Alerting and Notification Agent (Refactored from Customer Service Staffing Agent)
alerting_notification_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for sending automated alerts and notifications to relevant personnel regarding predicted failures, anomalies, or scheduled maintenance.",
    name="alerting-notification-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "send_alert_notification",
                "description": "Send a critical alert regarding a predicted failure or anomaly to a specified recipient group.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "recipient_group": {"type": "string", "description": "Target recipient group (e.g., 'maintenance team', 'operations manager')."},
                        "alert_message": {"type": "string", "description": "Content of the alert message."},
                        "severity": {"type": "string", "description": "Severity of the alert (e.g., 'urgent', 'high', 'medium')."}
                    },
                    "required": ["recipient_group", "alert_message", "severity"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "send_maintenance_reminder",
                "description": "Send a reminder for upcoming scheduled maintenance.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Identifier of the equipment."},
                        "scheduled_date": {"type": "string", "description": "Date of the upcoming maintenance in ISO-MM-DD format."}
                    },
                    "required": ["equipment_id", "scheduled_date"]
                }
            }
        }
    ]
)
print(f"Alerting and Notification Agent '{alerting_notification_agent.name}' created with ID: {alerting_notification_agent.id}")


# 8. Root Cause Analysis Agent (Refactored from Customer Communication Agent)
root_cause_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for assisting in identifying the underlying causes of equipment failures based on diagnostic data and historical context.",
    name="root-cause-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_failure_data",
                "description": "Analyze sensor data, event logs, and operational history to determine the root cause of a specific equipment failure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equipment_id": {"type": "string", "description": "Identifier of the failed equipment."},
                        "failure_event_data_json": {"type": "string", "description": "JSON string of data points leading up to the failure (e.g., sensor readings, error codes)."}
                    },
                    "required": ["equipment_id", "failure_event_data_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "suggest_corrective_actions",
                "description": "Suggest corrective actions based on a identified root cause of equipment failure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "root_cause_description": {"type": "string", "description": "Description of the identified root cause."},
                        "failure_type": {"type": "string", "description": "Type of failure (e.g., 'mechanical', 'electrical')."}
                    },
                    "required": ["root_cause_description", "failure_type"]
                }
            }
        }
    ]
)
print(f"Root Cause Analysis Agent '{root_cause_analysis_agent.name}' created with ID: {root_cause_analysis_agent.id}")


# 9. Supplier Coordination Agent (Refactored from Bank Operations Agent)
supplier_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for coordinating with spare parts suppliers, tracking orders, and managing delivery logistics.",
    name="supplier-coordination-agent",
    tools=[
        {"type": "web_search"}, # For looking up supplier contact info or external logistics updates.
        {
            "type": "function",
            "function": {
                "name": "track_part_order",
                "description": "Track the current status and estimated delivery of a specific spare part order.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "order_id": {"type": "string", "description": "Unique order identifier (e.g., 'PO-2024-001')."}
                    },
                    "required": ["order_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "expedite_order",
                "description": "Request to expedite an existing spare part order, citing urgency.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "order_id": {"type": "string", "description": "Unique order identifier."},
                        "reason_for_urgency": {"type": "string", "description": "Reason why the order needs to be expedited (e.g., 'critical equipment down')."}
                    },
                    "required": ["order_id", "reason_for_urgency"]
                }
            }
        }
    ]
)
print(f"Supplier Coordination Agent '{supplier_coordination_agent.name}' created with ID: {supplier_coordination_agent.id}")


print("\nAll new predictive maintenance-related agents have been defined.")

# --- Mock functions for predictive maintenance domain tools ---
# These mock functions provide simulated responses for the new agents' tools.

def get_sensor_data(equipment_id: str, sensor_type: str):
    """MOCK function to get sensor data."""
    print(f"\n[DEBUG] MOCK CALL: get_sensor_data - Equipment: {equipment_id}, Sensor: {sensor_type}")
    if equipment_id == "Pump-001" and sensor_type == "vibration":
        return {"status": "success", "data": 2.5, "unit": "mm/s", "timestamp": time.time(), "message": "Current vibration data."}
    if equipment_id == "Motor-A" and sensor_type == "temperature":
        return {"status": "success", "data": 85.2, "unit": "C", "timestamp": time.time(), "message": "Current motor temperature."}
    return {"status": "not_found", "message": "No data found for specified sensor/equipment."}

def log_equipment_status(equipment_id: str, status: str):
    """MOCK function to log equipment status."""
    print(f"\n[DEBUG] MOCK CALL: log_equipment_status - Equipment: {equipment_id}, Status: {status}")
    return {"status": "logged", "equipment_id": equipment_id, "new_status": status, "timestamp": time.time()}

def detect_anomalies(equipment_id: str, data_series_json: str):
    """MOCK function to detect anomalies."""
    print(f"\n[DEBUG] MOCK CALL: detect_anomalies for {equipment_id} with data: {data_series_json}")
    try:
        data_series = json.loads(data_series_json)
        anomalies = []
        # Simple anomaly detection: if value exceeds a threshold
        for item in data_series:
            if item.get("value", 0) > 10.0: # Example threshold
                anomalies.append({"timestamp": item.get("timestamp"), "value": item.get("value"), "reason": "Value above threshold"})
        return {"status": "success", "anomalies_found": bool(anomalies), "details": anomalies or "No anomalies detected."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for data series."}

def predict_component_failure(component_id: str, current_health_metrics_json: str):
    """MOCK function to predict component failure."""
    print(f"\n[DEBUG] MOCK CALL: predict_component_failure for {component_id} with metrics: {current_health_metrics_json}")
    try:
        metrics = json.loads(current_health_metrics_json)
        # Dummy prediction logic
        if component_id == "Bearing-X" and metrics.get("vibration", 0) > 1.5:
            return {
                "component_id": component_id,
                "predicted_failure_date": "2024-07-25",
                "confidence_score": 0.85,
                "recommendations": "Schedule replacement within 2 weeks. Order new bearing.",
                "flagged_metrics": {"vibration": "High"}
            }
        return {
            "component_id": component_id,
            "predicted_failure_date": "N/A",
            "confidence_score": 0.0,
            "recommendations": "No immediate failure predicted. Continue monitoring.",
            "flagged_metrics": {}
        }
    except json.JSONDecodeError:
        return {
            "component_id": component_id,
            "predicted_failure_date": "Error",
            "confidence_score": 0.0,
            "recommendations": "Invalid metrics data.",
            "flagged_metrics": {}
        }

def schedule_preventative_maintenance(component_id: str, scheduled_date: str, priority: str):
    """MOCK function to schedule preventative maintenance."""
    print(f"\n[DEBUG] MOCK CALL: schedule_preventative_maintenance - Component: {component_id}, Date: {scheduled_date}, Priority: {priority}")
    return {"status": "scheduled", "task_id": "MAINT-001", "component": component_id, "date": scheduled_date}

def get_maintenance_team_availability(team_type: str, date: str):
    """MOCK function to get maintenance team availability."""
    print(f"\n[DEBUG] MOCK CALL: get_maintenance_team_availability for {team_type} on {date}")
    if team_type.lower() == "mechanical" and date == "2024-07-20":
        return {"status": "success", "available_teams": ["Team-Alpha", "Team-Beta"]}
    return {"status": "success", "available_teams": []}

def check_part_stock(part_number: str):
    """MOCK function to check part stock."""
    print(f"\n[DEBUG] MOCK CALL: check_part_stock for {part_number}")
    if part_number == "XYZ-456":
        return {"status": "success", "part_number": part_number, "stock_level": 5, "min_stock": 2}
    return {"status": "not_found", "message": "Part not found."}

def order_spare_part(part_number: str, quantity: int, supplier_id: str):
    """MOCK function to order spare part."""
    print(f"\n[DEBUG] MOCK CALL: order_spare_part - Part: {part_number}, Qty: {quantity}, Supplier: {supplier_id}")
    return {"status": "order_placed", "order_id": "PO-2024-001", "part": part_number, "quantity": quantity}

def get_equipment_maintenance_history(equipment_id: str):
    """MOCK function to retrieve equipment maintenance history."""
    print(f"\n[DEBUG] MOCK CALL: get_equipment_maintenance_history for {equipment_id}")
    if equipment_id == "Pump-001":
        return {"status": "success", "history": [
            {"date": "2023-10-01", "type": "preventative", "description": "Lubrication"},
            {"date": "2024-03-15", "type": "repair", "description": "Bearing replacement"}
        ]}
    return {"status": "not_found", "message": "No history found."}

def search_repair_logs(search_query: str):
    """MOCK function to search repair logs."""
    print(f"\n[DEBUG] MOCK CALL: search_repair_logs for query: {search_query}")
    if "bearing noise" in search_query.lower():
        return {"status": "success", "results": ["Log entry 1: Pump-001 bearing noise, replaced.", "Log entry 2: Motor-B intermittent bearing noise."]}
    return {"status": "success", "results": [], "message": "No matching logs found."}

def send_alert_notification(recipient_group: str, alert_message: str, severity: str):
    """MOCK function to send an alert notification."""
    print(f"\n[DEBUG] MOCK CALL: send_alert_notification - Group: {recipient_group}, Msg: '{alert_message}', Severity: {severity}")
    return {"status": "sent", "recipient": recipient_group, "message": alert_message, "severity": severity}

def send_maintenance_reminder(equipment_id: str, scheduled_date: str):
    """MOCK function to send a maintenance reminder."""
    print(f"\n[DEBUG] MOCK CALL: send_maintenance_reminder for {equipment_id} on {scheduled_date}")
    return {"status": "sent", "equipment_id": equipment_id, "reminder_date": scheduled_date}

def analyze_failure_data(equipment_id: str, failure_event_data_json: str):
    """MOCK function to analyze failure data for root cause."""
    print(f"\n[DEBUG] MOCK CALL: analyze_failure_data for {equipment_id} with data: {failure_event_data_json}")
    try:
        failure_data = json.loads(failure_event_data_json)
        # Simple root cause analysis based on mock data.
        if equipment_id == "Compressor-C" and failure_data.get("pressure_spike_before_failure"):
            return {"status": "success", "root_cause": "Sudden pressure spike leading to seal failure.", "suggested_fix": "Inspect pressure regulator."}
        return {"status": "success", "root_cause": "Insufficient data to determine root cause.", "suggested_fix": "Gather more diagnostic data."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON for failure event data."}

def suggest_corrective_actions(root_cause_description: str, failure_type: str):
    """MOCK function to suggest corrective actions."""
    print(f"\n[DEBUG] MOCK CALL: suggest_corrective_actions - Root Cause: '{root_cause_description}', Type: {failure_type}")
    if "pressure spike" in root_cause_description.lower():
        return {"status": "success", "actions": ["Calibrate pressure sensors.", "Check pressure relief valve.", "Review operational parameters."]}
    return {"status": "success", "actions": ["Standard troubleshooting procedure.", "Consult equipment manual."], "message": "General corrective actions."}

def track_part_order(order_id: str):
    """MOCK function to track part order."""
    print(f"\n[DEBUG] MOCK CALL: track_part_order for {order_id}")
    if order_id == "PO-2024-001":
        return {"status": "in_transit", "estimated_delivery": "2024-07-05", "tracking_info": "XYZ789", "message": "Order is en route."}
    return {"status": "not_found", "message": "Order not found."}

def expedite_order(order_id: str, reason_for_urgency: str):
    """MOCK function to expedite order."""
    print(f"\n[DEBUG] MOCK CALL: expedite_order - Order: {order_id}, Reason: '{reason_for_urgency}'")
    return {"status": "expedite_requested", "order_id": order_id, "message": "Expedite request sent to supplier."}


# --- Master Tool Executor Mapping (Updated for Predictive Maintenance Domain) ---
tool_executor = {
    "get_sensor_data": get_sensor_data,
    "log_equipment_status": log_equipment_status,
    "detect_anomalies": detect_anomalies,
    "predict_component_failure": predict_component_failure,
    "schedule_preventative_maintenance": schedule_preventative_maintenance,
    "get_maintenance_team_availability": get_maintenance_team_availability,
    "check_part_stock": check_part_stock,
    "order_spare_part": order_spare_part,
    "get_equipment_maintenance_history": get_equipment_maintenance_history,
    "search_repair_logs": search_repair_logs,
    "send_alert_notification": send_alert_notification,
    "send_maintenance_reminder": send_maintenance_reminder,
    "analyze_failure_data": analyze_failure_data,
    "suggest_corrective_actions": suggest_corrective_actions,
    "track_part_order": track_part_order,
    "expedite_order": expedite_order,
    "internal_web_search_tool": lambda *args, **kwargs: "Mock web search: General industrial information retrieved."
}

# --- Function to standardize tools for client.chat.complete (remains the same) ---
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# --- Test Case Execution for Predictive Maintenance Agents ---

print("\n--- Executing Test Cases for Predictive Maintenance Agents (via chat completions) ---")

test_cases = [
    {
        "agent": equipment_monitoring_agent,
        "name": "Equipment Monitoring Agent",
        "query": "Get vibration data for Pump-001.",
        "expected_tool_call": "get_sensor_data"
    },
    {
        "agent": equipment_monitoring_agent,
        "name": "Equipment Monitoring Agent",
        "query": "Log status of Compressor-C as 'running'.",
        "expected_tool_call": "log_equipment_status"
    },
    {
        "agent": anomaly_detection_agent,
        "name": "Anomaly Detection Agent",
        "query": "Detect anomalies in Motor-A's temperature readings: [{\"timestamp\": \"2024-06-21T10:00:00\", \"value\": 80.1}, {\"timestamp\": \"2024-06-21T10:05:00\", \"value\": 95.5}, {\"timestamp\": \"2024-06-21T10:10:00\", \"value\": 105.2}]",
        "expected_tool_call": "detect_anomalies"
    },
    {
        "agent": failure_prediction_agent,
        "name": "Failure Prediction Agent",
        "query": "Predict failure for Bearing-X with current health metrics: {\"vibration\": 1.8, \"temp\": 60}",
        "expected_tool_call": "predict_component_failure"
    },
    {
        "agent": maintenance_scheduling_agent,
        "name": "Maintenance Scheduling Agent",
        "query": "Schedule preventative maintenance for Motor-A on 2024-07-20 with high priority.",
        "expected_tool_call": "schedule_preventative_maintenance"
    },
    {
        "agent": maintenance_scheduling_agent,
        "name": "Maintenance Scheduling Agent",
        "query": "Check availability of mechanical maintenance teams on 2024-07-20.",
        "expected_tool_call": "get_maintenance_team_availability"
    },
    {
        "agent": parts_inventory_agent,
        "name": "Parts Inventory Agent",
        "query": "Check stock level for part XYZ-456.",
        "expected_tool_call": "check_part_stock"
    },
    {
        "agent": parts_inventory_agent,
        "name": "Parts Inventory Agent",
        "query": "Order 10 units of part ABC-123 from Supplier-Gamma.",
        "expected_tool_call": "order_spare_part"
    },
    {
        "agent": maintenance_history_agent,
        "name": "Maintenance History Agent",
        "query": "Get maintenance history for Pump-001.",
        "expected_tool_call": "get_equipment_maintenance_history"
    },
    {
        "agent": maintenance_history_agent,
        "name": "Maintenance History Agent",
        "query": "Search repair logs for 'bearing noise'.",
        "expected_tool_call": "search_repair_logs"
    },
    {
        "agent": alerting_notification_agent,
        "name": "Alerting and Notification Agent",
        "query": "Send urgent alert to maintenance team: 'Critical vibration anomaly detected in Pump-001'.",
        "expected_tool_call": "send_alert_notification"
    },
    {
        "agent": alerting_notification_agent,
        "name": "Alerting and Notification Agent",
        "query": "Send reminder for upcoming maintenance on Motor-A, scheduled for 2024-07-20.",
        "expected_tool_call": "send_maintenance_reminder"
    },
    {
        "agent": root_cause_analysis_agent,
        "name": "Root Cause Analysis Agent",
        "query": "Analyze failure data for Compressor-C: {\"pressure_spike_before_failure\": true, \"temp_at_failure\": 120, \"error_code\": \"E-005\"}",
        "expected_tool_call": "analyze_failure_data"
    },
    {
        "agent": root_cause_analysis_agent,
        "name": "Root Cause Analysis Agent",
        "query": "Suggest corrective actions for 'Sudden pressure spike' leading to 'mechanical' failure.",
        "expected_tool_call": "suggest_corrective_actions"
    },
    {
        "agent": supplier_coordination_agent,
        "name": "Supplier Coordination Agent",
        "query": "Track order PO-2024-001.",
        "expected_tool_call": "track_part_order"
    },
    {
        "agent": supplier_coordination_agent,
        "name": "Supplier Coordination Agent",
        "query": "Expedite order PO-2024-001 due to critical equipment down.",
        "expected_tool_call": "expedite_order"
    },
    {
        "agent": anomaly_detection_agent, # Reusing for web search example
        "name": "Anomaly Detection Agent (Web Search Example)",
        "query": "Best practices for predictive maintenance data collection.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"] # This is for tracking, not direct control.

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        # Get the API-compatible tool list for the current agent.
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        print(f" [DEBUG] Sending initial user query to the {agent_name}...")
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                # Check if the proposed tool exists in our local executor mapping.
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        # Execute the local mock function based on its name.
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                # Add the tool output message to the conversation history.
                # This is crucial for the model to "see" the result of the tool call.
                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id # Links the tool output to the specific call request.
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            # Second turn: Send the conversation history (including tool outputs) back to the model.
            # The model will then generate a final response based on the tool's output.
            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list, # Tools must be provided in all calls if they are part of the conversation context.
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            # Add the final assistant response to history (optional for a single-turn demo, but good practice).
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)

        else:
            # If no tool calls were proposed in the first turn, print the direct response.
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")
        print("If you continue to experience errors, a complete restart of your Python environment (e.g., Colab runtime) might help.")

print("\n--- All test cases execution complete. ---")


Mounted at /content/gdrive
Creating AI agents for Predictive Maintenance Domain...
Equipment Monitoring Agent 'equipment-monitoring-agent' created with ID: ag_06856b87fde773438000b6fd03835f3e
Anomaly Detection Agent 'anomaly-detection-agent' created with ID: ag_06856b880102790c8000417a199b9471
Failure Prediction Agent 'failure-prediction-agent' created with ID: ag_06856b8803d673ff8000e35e3d6434ef
Maintenance Scheduling Agent 'maintenance-scheduling-agent' created with ID: ag_06856b8806dd77f080004e215f0929ca
Parts Inventory Agent 'parts-inventory-agent' created with ID: ag_06856b8809b678c38000eccc9fa8dc20
Maintenance History Agent 'maintenance-history-agent' created with ID: ag_06856b880cb472688000b70fb8eff800
Alerting and Notification Agent 'alerting-notification-agent' created with ID: ag_06856b880fa77f968000c3c826caa94a
Root Cause Analysis Agent 'root-cause-analysis-agent' created with ID: ag_06856b8812927eec800018f62b0ee90f
Supplier Coordination Agent 'supplier-coordination-agent' c